In [ ]:
# %pip install bayesian-optimization
%pip install dataclasses
%pip install matching-ds-tools

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import json
import datetime
import re
# from bayes_opt import BayesianOptimization
import torch
from mini_sim.DQN_offlineData import *

import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from queryrunner_client import Client
USER_EMAIL = 'ssadeghi@uber.com'
qclient = Client(user_email=USER_EMAIL)
CONSUMER_NAME = 'intelligentdispatch'

import os
import warnings
warnings.filterwarnings('ignore')
import multiprocessing
from joblib import Parallel, delayed
#num_cores = multiprocessing.cpu_count()  -- 48
n_cores = 4

In [ ]:
from dataclasses import dataclass
import itertools
from typing import *
import numpy as np
import pandas as pd
from queryrunner_client import Client as QRClient
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
import mdstk
from mdstk.data_fetcher.data_fetcher import DataFetcher
from mdstk.data_fetcher.cached_data_fetcher import CachedDataFetcher
pd.set_option('display.max_columns', None)

In [ ]:
"""
-- San Francisco
prefix = 'LSR'
hex_digits = '34'

city_id = 1
vvid = 8

datestrs = [  # 1 week
    '2022-11-11',
    '2022-11-12',
    '2022-11-13',
    '2022-11-14',
    '2022-11-15',
    '2022-11-16',
    '2022-11-17',
]

MAX_LAT = 38.19
MIN_LAT = 37.09
MAX_LNG = -121.55
MIN_LNG = -122.60

-- Detroit
prefix = 'LSR'
hex_digits = '34'

city_id = 50
vvid = 425

datestrs = [  # 1 week
    '2022-11-11',
    '2022-11-12',
    '2022-11-13',
    '2022-11-14',
    '2022-11-15',
    '2022-11-16',
    '2022-11-17',
]

MAX_LAT = 42.89
MIN_LAT = 42.01
MAX_LNG = -82.68
MIN_LNG = -83.93


-- Philadelphia
prefix = 'LSR'
hex_digits = '34'

city_id = 20
vvid = 663

datestrs = [  # 1 week
    '2022-11-11',
    '2022-11-12',
    '2022-11-13',
    '2022-11-14',
    '2022-11-15',
    '2022-11-16',
    '2022-11-17',
]

MAX_LAT = 40.22
MIN_LAT = 39.74
MAX_LNG = -74.86
MIN_LNG = -75.46

"""

In [ ]:
## INPUTS 

prefix = 'LSR'
hex_digits = '34'

city_id = 50
vvid = 425

datestrs = [  # 1 week
    '2022-11-11',
    '2022-11-12',
    '2022-11-13',
    '2022-11-14',
    '2022-11-15',
    '2022-11-16',
    '2022-11-17',
]

MAX_LAT = 42.89
MIN_LAT = 42.01
MAX_LNG = -82.68
MIN_LNG = -83.93

DISCOUNT = 0.995
VALUE_DEGRADE_LEVEL = 0.85

In [ ]:
# data collection

QUERY = """
with dispatch as (
select 
    datestr,
    msg.cityid,
    msg.ctplangenrequestuuid as plangen_uuid,
    msg.ctrequestuuid as scan_uuid,
    j as job_uuid,
    msg.supplyuuid,
    msg.planactiontype
from 
    rawdata_user.kafka_hp_multileg_dispatched_plan_nodedup
cross join 
    unnest(msg.jobuuid) jobs(j)
where 
    datestr = '{datestr}'
    and msg.cityid = {city_id}
    and msg.vehicleviewid = {vvid} 
    and msg.tenancy = 'uber/production'
    and CARDINALITY(msg.jobuuid) > 0
    and substr(msg.ctrequestuuid, 1, length('{digits}')) = '{digits}'
),
plangen as (
select 
    msg.scanuuid as plangen_uuid, 
    p.uuid as job_uuid,
    j.supplyuuid
from 
    rawdata_user.kafka_hp_multileg_matching_observability_proposals_v2_nodedup
cross join 
    unnest(msg.proposals) as job(j)
cross join 
    unnest(j.jobs) as plan(p)
where 
    datestr = '{datestr}'
    and msg.cityid = {city_id}
    and msg.flowtype = 'solo_batch'
    and msg.tenancy = 'uber/production'
    and j.status = 'eligible'
),
mgv as (
select datestr,
    msg.city_id,
    msg.job_uuid,
    msg.client_uuid,
    msg.ct_request_uuid as plangen_uuid,
    msg.supply_uuid,
    msg.supply_plan_uuid as plan_uuid,
    msg.unadjusted_eta as eta,
    (CASE
      WHEN msg.adjustedeta > 1500 THEN 1500.0
      WHEN msg.adjustedeta < 0 THEN 0.0
      ELSE msg.adjustedeta
    END) as adjustedeta,
    round(msg.job_surge, 4) as surge_mul,
    round(msg.eventual_completion_probability, 4) as eventual_comp_prob,
    msg.ranking_metric,
    round(1 - msg.solo_cancel_model_driver_accept_prob, 4) as d_proba,
    round(1 - msg.solo_cancel_model_rider_accept_prob, 4) as r_proba,
    round(1 - msg.spinner_survive_prob_before_next_scan, 4) as s_proba,
    msg.preferred_destination_adjustment,
    msg.objective_value as of_value,
    msg.inconvenience_etd - msg.ranking_metric as trip_length,
    msg.dropoff_latitude as dropoff_latitude,
    msg.dropoff_longitude as dropoff_longitude,
    msg.pickup_latitude as pickup_latitude,
    msg.pickup_longitude as pickup_longitude 
from   
    rawdata.kafka_hp_multileg_mgv_log_nodedup
where  
    datestr = '{datestr}'
    and msg.city_id = {city_id}
    and msg.tenancy = 'uber/production'
    and msg.vehicle_view_id = {vvid} 
    and msg.flow_type = 'solo_batch'
    and msg.job_uuid <> msg.client_uuid
    and msg.calculator_type = 'markov_eta_v2'
),
geoloc as (
    SELECT
    datestr as datestr,
    msg.cityid as city_id,
    msg.etasec as eta,
    msg.flowtype as flow_type,
    msg.timestampms as ts,
    msg.supplyuuid as supply_uuid,
    originalsupplywaypoints.latitude as driver_origin_lat,
    originalsupplywaypoints.longitude as driver_origin_lng,
    jobs.uuid as job_uuid,
    jobs.timetodropoffinplaninsecs as timetodropoffinplaninsecs
    FROM
    rawdata_user.kafka_hp_plangenerator_matching_plans_log_nodedup
    CROSS JOIN
    UNNEST(msg.originalsupplywaypoints) as t(originalsupplywaypoints)
    CROSS JOIN
    UNNEST(msg.jobs) as j(jobs)
    WHERE
    datestr='{datestr}'
    and msg.cityid={city_id}
),
test as (
select
    mgv.datestr,
    mgv.city_id,
    dispatch.scan_uuid,
    mgv.plangen_uuid,
    mgv.job_uuid,
    dispatch.planactiontype,
    mgv.supply_uuid,
    case when dispatch.supplyuuid = mgv.supply_uuid then 1 else 0 end as is_selected,
    mgv.eta,
    mgv.adjustedeta,
    geoloc.timetodropoffinplaninsecs,
    geoloc.driver_origin_lat,
    geoloc.driver_origin_lng,
    mgv.pickup_latitude,
    mgv.pickup_longitude,
    mgv.dropoff_latitude,
    mgv.dropoff_longitude,
    great_circle_distance(
        mgv.pickup_latitude, mgv.pickup_longitude,
        mgv.dropoff_latitude, mgv.dropoff_longitude
      ) AS haversine_dist_km,
    round(POWER(1 - mgv.adjustedeta / 1500.0, 1), 4) as eta_one,
    round(POWER(1 - mgv.adjustedeta / 1500.0, 1.05), 4) as eta_one_five,
    round(POWER(1 - mgv.adjustedeta / 1500.0, 1.10), 4) as eta_one_ten,
    round(POWER(1 - mgv.adjustedeta / 1500.0, 1.15), 4) as eta_one_fifteen,
    round(POWER(1 - mgv.adjustedeta / 1500.0, 1.20), 4) as eta_one_twenty,
    round(POWER(1 - mgv.adjustedeta / 1500.0, 1.25), 4) as eta_one_quarter,
    mgv.surge_mul,
    mgv.eventual_comp_prob,
    round(1.0 / (1.0 + POWER(mgv.surge_mul, 2)), 4) as network_contention_2,
    round(1.0 / (1.0 + POWER(mgv.surge_mul, 3)), 4) as network_contention_3,
    round(1.0 / (1.0 + POWER(mgv.surge_mul, 5)), 4) as network_contention_5,
    mgv.ranking_metric,
    mgv.d_proba,
    mgv.r_proba,
    mgv.s_proba,
    round((1.0 - mgv.d_proba) * (1.0 - mgv.r_proba) * (1.0 - mgv.s_proba) + mgv.eventual_comp_prob * mgv.d_proba, 4) as cr_ratio,
    round((1.0 - mgv.d_proba) * (1.0 - mgv.r_proba) + mgv.eventual_comp_prob * mgv.d_proba, 4) as crof_ratio,
    mgv.preferred_destination_adjustment,
    mgv.of_value,
    mgv.trip_length,
    fare.est_rider_quoted_final_fare as fare,
    fare.est_rider_quoted_final_fare * 1.0 / fare.usd_fx_rate as fare_usd,
    fare.request_timestamp_local as local_time
from
    mgv
join
    plangen
on 
    mgv.plangen_uuid = plangen.plangen_uuid
    and mgv.job_uuid = plangen.job_uuid
    and mgv.supply_uuid = plangen.supplyuuid
join
    dispatch
on
    mgv.plangen_uuid = dispatch.plangen_uuid
    and mgv.job_uuid = dispatch.job_uuid
join
    dwh.fact_trip_fare fare 
on
    mgv.job_uuid = fare.trip_uuid
    and fare.datestr = '{datestr}'
    and fare.city_id = {city_id}
join
    geoloc
on
    mgv.job_uuid = geoloc.job_uuid    
    and mgv.supply_uuid = geoloc.supply_uuid
)
select * from test
LIMIT 2000000
"""

In [ ]:
@dataclass
class Query:
    prefix: str
    hex_digits: str
    city_id: int
    vvid: str
    datestr: str
    
    def __post_init__(self):
        self.name = f'{self.prefix}_city{self.city_id}_{self.vvid}_{self.datestr}_segment{self.hex_digits}'
        self.qry = QUERY.format(city_id=self.city_id, vvid=self.vvid, digits=self.hex_digits, datestr=self.datestr)

In [ ]:
class MyDataFetcher(DataFetcher):
    def query_many_presto(self, *args, **kwargs):
        return super().query_many_presto(*args, **kwargs)

In [ ]:
def add_origin_SV(row, term = 'mean'):
    row['local_time'] = pd.to_datetime(row['local_time'])
    weekday = row['local_time'].weekday()
    time = row['local_time'].hour * 3600 + \
           row['local_time'].minute * 60 + \
           row['local_time'].second
    if time > 24 * 3600:
        time = time % (24 * 3600)    
        weekday = (weekday + 1) % 7
    lat = row.driver_origin_lat
    lng = row.driver_origin_lng
    inp = torch.tensor([weekday, time, lat, lng], dtype=torch.float).reshape(1,-1)
    LTSV.eval()
    if term == 'mean':
        out,_ = LTSV(inp)
    elif term == 'variance':
        _, out = LTSV(inp)
    else:
        raise Exception("DNN output term invalid")
    return out.item()

def add_pickup_SV(row, term = 'mean'):
    row['local_time'] = pd.to_datetime(row['local_time'])
    weekday = row['local_time'].weekday()
    time = row['local_time'].hour * 3600 + \
           row['local_time'].minute * 60 + \
           row['local_time'].second + row['adjustedeta']
    if time > 24 * 3600:
        time = time % (24 * 3600)    
        weekday = (weekday + 1) % 7
    lat = row.pickup_latitude
    lng = row.pickup_longitude
    inp = torch.tensor([weekday, time, lat, lng], dtype=torch.float).reshape(1,-1)
    LTSV.eval()
    if term == 'mean':
        out,_ = LTSV(inp)
    elif term == 'variance':
        _, out = LTSV(inp)
    else:
        raise Exception("DNN output term invalid")
    return out.item()

def add_destination_SV(row, term = 'mean'):
    row['local_time'] = pd.to_datetime(row['local_time'])
    weekday = row['local_time'].weekday()
    time = row['local_time'].hour * 3600 + \
           row['local_time'].minute * 60 + \
           row['local_time'].second + row['timetodropoffinplaninsecs']
    if time > 24 * 3600:
        time = time % (24 * 3600)
        weekday = (weekday + 1) % 7
    lat = row.dropoff_latitude
    lng = row.dropoff_longitude
    inp = torch.tensor([weekday, time, lat, lng], dtype=torch.float).reshape(1,-1)
    LTSV.eval()
    if term == 'mean':
        out,_ = LTSV(inp)
    elif term == 'variance':
        _, out = LTSV(inp)
    else:
        raise Exception("DNN output term invalid")
    return out.item()


# def batch_origin_SV(df):
#     df['local_time'] = pd.to_datetime(df['local_time'])
#     time = df['local_time'].hour * 3600 + \
#            df['local_time'].minute * 60 + \
#            df['local_time'].second + df['timetodropoffinplaninsecs']
    

In [ ]:

# Calculate new objective function
def clean_df(df):
    df = df[df['fare'].notnull()]
    df['trip_length'][df['trip_length'] <= 100] = 100
    df = df.drop_duplicates(subset=['job_uuid', 'supply_uuid'])
    df = df.dropna()
    df['origin_SV_mean'] = df.apply(lambda row : add_origin_SV(row, term = 'mean'), axis = 1)
    df['origin_SV_var'] = df.apply(lambda row : add_origin_SV(row, term = 'variance'), axis = 1)
    df['pickup_SV_mean'] = df.apply(lambda row : add_pickup_SV(row, term = 'mean'), axis = 1)
    df['pickup_SV_var'] = df.apply(lambda row : add_pickup_SV(row, term = 'variance'), axis = 1)
    df['dest_SV_mean'] = df.apply(lambda row : add_destination_SV(row, term = 'mean'), axis = 1)
    df['dest_SV_var'] = df.apply(lambda row : add_destination_SV(row, term = 'variance'), axis = 1)
    return df

# local solver
def solve_dict(
    scan: dict, 
    cost_col: str, 
    job_singleton: float = 1500,
    infinity: float = 1000000
):
    job_list = list(set([k[0] for k in scan.keys()]))
    job_idx = {j: i for i, j in enumerate(job_list)}
    job_count = len(job_list)

    supply_list = list(set([k[1] for k in scan.keys()]))
    supply_idx = {s: i for i, s in enumerate(supply_list)}
    supply_count = len(supply_list)
    
    utility = np.full((len(job_list), len(supply_list) + len(job_list)), infinity, dtype=np.float32)
    for k in scan.keys():
        jidx = job_idx[k[0]]
        sidx = supply_idx[k[1]]
        utility[jidx, sidx] = scan[k][cost_col]
    for i in range(len(job_list)):
        utility[i, supply_count + i] = job_singleton
            
    # solve
    job_sol, supply_sol = linear_sum_assignment(utility)

    result = set()
    for jidx, sidx in zip(job_sol, supply_sol):
        j = job_list[jidx]
        if sidx >= supply_count:
            result.add((j,))
        else:
            s = supply_list[sidx]
            result.add((j, s))
            
    assert len(result) == len(job_list)
    return result  

In [ ]:
from dataclasses import field

@dataclass
class ScanMetrics:
    total_jobs: int = 0.
    total_eta: float = 0.
    total_offer: float = 0.
    total_ar: float = 0.
    total_rc: float = 0.
    total_trip: float = 0.
    total_gb: float = 0.
    total_fare: float = 0.
    total_overwrite: int = 0.
    total_jobsurge: int = 0.
    list_etas: list = field(default_factory = list)
    
    def __add__(self, o: 'ScanMetrics') -> 'ScanMetrics':
        return ScanMetrics(
            self.total_jobs + o.total_jobs,
            self.total_eta + o.total_eta,
            self.total_offer + o.total_offer,
            self.total_ar + o.total_ar,
            self.total_rc + o.total_rc,
            self.total_trip + o.total_trip,
            self.total_overwrite + o.total_overwrite,
            self.total_gb + o.total_gb,
            self.total_fare + o.total_fare,
            self.total_jobsurge + o.total_jobsurge,
            self.list_etas.expand + o.list_etas
        )
    def __iadd__(self, o: 'ScanMetrics') -> 'ScanMetrics':
        self.total_jobs += o.total_jobs
        self.total_eta += o.total_eta
        self.total_offer += o.total_offer
        self.total_ar += o.total_ar
        self.total_rc += o.total_rc
        self.total_trip += o.total_trip
        self.total_overwrite += o.total_overwrite
        self.total_gb += o.total_gb
        self.total_fare += o.total_fare
        self.total_jobsurge += o.total_jobsurge
        self.list_etas += o.list_etas
        
        return self

In [ ]:
# Metric Summary
def metric_summary_dict(
    scan_dict: Dict[str, Dict[str, Any]],
    matching: set, 
    overwrite: int,
) -> ScanMetrics:
    sm = ScanMetrics()
    sm.total_jobs = len(matching)
    sm.total_overwrite = overwrite
    
    for m in matching:
        if len(m) == 2:
            row = scan_dict[(m[0], m[1])]
            sm.total_offer += 1
            sm.total_eta += row['eta']
            sm.total_ar += 1 - row['d_proba']
            sm.total_rc += row['r_proba']
            sm.total_jobsurge += row['surge_mul']
            if row['trip_length'] < 7200:
                sm.total_trip += row['trip_length']
            if row['fare_usd'] > 0:
                sm.total_gb += (1 - row['d_proba']) * (1 - row['r_proba']) * row['fare_usd']
                sm.total_fare += row['fare_usd']
                
            sm.list_etas.append(row['eta'])

    return sm

def solve_all_dict(df, solver: Callable[[dict], set]):
    total_scans = dict(tuple(df.groupby('scan_uuid')))

    sm = ScanMetrics()
    overwrite_summary = {}
    for scan_uuid, scan_df in total_scans.items():
        scan = (scan_df.set_index(['job_uuid', 'supply_uuid']).to_dict(orient='index'))
        matching, overwrite, out = solver(scan)
        overwrite_summary[scan_uuid] = out
        sm += metric_summary_dict(scan, matching, overwrite)
        
    return {'total_jobs': round(sm.total_jobs),
            'match_rate': round(sm.total_offer * 1.0 / sm.total_jobs, 3),
            'overwrite': round(sm.total_overwrite * 1.0 / sm.total_jobs, 3), # different decisions compared to Markov
            'Average Matched ETA': round(sm.total_eta * 1.0 / sm.total_offer, 2),
            'P90 Matched ETA': round(np.percentile(sm.list_etas, 90), 2),
            'Driver AR': round(sm.total_ar * 1.0 / sm.total_offer, 3),
            'Rider cancel': round(sm.total_rc * 1.0 / sm.total_offer, 3),
            'Average trip length': round(sm.total_trip * 1.0 / sm.total_offer, 2),
#             'Average Matched Fare': round(sm.total_fare * 1.0 / sm.total_offer, 2),
            'Average Matched Fare': round(sm.total_gb * 1.0 / sm.total_offer, 2),
            'Average Job Surge': round(sm.total_jobsurge * 1.0 / sm.total_offer, 2),
            'Total GB': round(sm.total_gb),
            'overwrite_summary': overwrite_summary,
           }

def different_matching_decision(m1,m2):
    return m1.difference(m2), m2.difference(m1)

def supply_cost_solve_dict(scan, is_markov = False, secondary_singleton = 0.0):
    # Markov
#     primary_matching = solve_dict(scan, 'of_value', job_singleton = 1500)
    primary_matching = solve_dict(scan, 'of_value_EFOF', job_singleton = 0)
    if is_markov:      
        return primary_matching, 0
    
    # SCA solve
    secondary_matching = solve_dict(scan, 'new_of', job_singleton = secondary_singleton)
    
    primary_dict = {}
    secondary_dict = {}
    # k[0] job and k[1] supply
    for k in primary_matching:
        try:
            primary_dict[k[1]] = k[0]
        except:
            pass

    for k in secondary_matching:
        try:
            secondary_dict[k[1]] = k[0]
        except:
            pass
            
            
    all_supplies = set(primary_dict.keys()).intersection(set(secondary_dict.keys()))
    all_jobs = set(primary_dict.values()).intersection(set(secondary_dict.values()))
    
    rev_primary_dict = {v:k for k,v in primary_dict.items()}
    rev_secondary_dict = {v:k for k,v in secondary_dict.items()}
    
#     show same supplies with different jobs (check dropoff values)
    out = []
    for k in all_supplies:
        if primary_dict[k] != secondary_dict[k]:
            out.append([k, primary_dict[k], secondary_dict[k]])

# #     show same jobs with different supplies (check origin values)
#     out = []
#     for k in all_jobs:
#         if rev_primary_dict[k] != rev_secondary_dict[k]:
#             out.append([k, rev_primary_dict[k], rev_secondary_dict[k]])
                
    different_matches = len(different_matching_decision(primary_matching, secondary_matching)[0])
    return secondary_matching, different_matches, out


In [ ]:


queries = [
    Query(prefix=prefix, hex_digits=hex_digits, city_id=city_id, vvid=vvid, datestr=datestr)
    for datestr in datestrs
]

cache_qry_map = {
    q.name: q.qry 
    for q in queries
}

cdf = CachedDataFetcher(
    data_fetcher=MyDataFetcher(
        user_email=USER_EMAIL,
        consumer_name=CONSUMER_NAME,
    ),
    cache_qry_map=cache_qry_map,
    datacenter='phx2',
    datasource='presto-secure',
)

cdf.fetch(bust_cache=False)

In [ ]:
LTSV = Net(MAX_LAT, MIN_LAT, MAX_LNG, MIN_LNG)
PATH = f'checkpoints/LTSV_city{city_id}_vvid{vvid}_gamma{DISCOUNT}_discount{VALUE_DEGRADE_LEVEL}'
LTSV.load_state_dict(torch.load(PATH))

In [ ]:
## 10156489
## 144925

In [ ]:
%%time
# clean data
scans = pd.concat(cdf.dfs.values(), axis=0, ignore_index=True) 
print(len(scans))
df = scans
df = clean_df(df)
print(len(df))
del scans

In [ ]:
"""
MarkovETA

{'total_jobs': 4613,
 'match_rate': 0.893,
 'overwrite': 0.0,
 'Average Matched ETA': 405.74,
 'P90 Matched ETA': 956.2,
 'Driver AR': 0.341,
 'Rider cancel': 0.08,
 'Average trip length': 911.02,
 'Average Matched Fare': 25.82,
 'Total GB': 28180}
 
EFOF
{'total_jobs': 4613,
 'match_rate': 0.909, 
 'overwrite': 0.341, 
 'Average Matched ETA': 429.62, 
 'P90 Matched ETA': 1036.6, 
 'Driver AR': 0.331, 
 'Rider cancel': 0.091, 
 'Average trip length': 980.51, 
 'Average Matched Fare': 27.83, 
 'Total GB': 30002}
"""

In [ ]:
eta_square = np.power(1 - df['adjustedeta'] / 1500.0, 2)
tripValue = df['fare']
OF = - eta_square * df['cr_ratio'] * tripValue
df['of_value_EFOF'] = OF
min_val = min(df['of_value_EFOF'])
max_val = max(df['of_value_EFOF'])
df['of_value_EFOF'] = -1 * (1 - (df['of_value_EFOF'] - min_val) / (max_val - min_val))

In [ ]:
df['dest_SV_var'].describe()

In [ ]:
def compute_new_of(df, coef):
    eta_square = np.power(1 - df['adjustedeta'] / 1500.0, 2)
    ## approach 1
    diffSV = df['dest_SV_mean'] / np.sqrt(df['dest_SV_var']) - df['origin_SV_mean'] / np.sqrt(df['origin_SV_var'])
    diffSV = diffSV.clip(-10,10)
    ## approach 2
#     diffSV = df['dest_SV_mean'] - df['origin_SV_mean']
#     diffSV[np.sqrt(df['origin_SV_var']) > 4] = 0
    tripValue = df['fare'] + coef * diffSV
    OF = - eta_square * df['cr_ratio'] * tripValue
    df['new_of'] = OF
    min_val = min(df['new_of'])
    max_val = max(df['new_of'])
    df['new_of'] = -1 * (1 - (df['new_of'] - min_val) / (max_val - min_val))
    return df

In [ ]:
df.head(1)

In [ ]:
res_dict = {'coef': [], 'match_rate': [], 'Average Matched ETA': [], 'Average Matched Fare': [], 'Total GB': [],
           'overwrite': [], 'Driver AR': [], 'Rider cancel': [], 'Average trip length': [], 'Average Job Surge': []}

for coef in np.arange(0,8.0,0.1):
# for coef in [0, 12.5]:
    df = compute_new_of(df, coef)
    out = solve_all_dict(df, lambda scan: supply_cost_solve_dict(scan, is_markov = False))
    res_dict['coef'].append(coef)
    res_dict['match_rate'].append(out['match_rate'])
    res_dict['Average Matched ETA'].append(out['Average Matched ETA'])
    res_dict['Average Matched Fare'].append(out['Average Matched Fare'])
    res_dict['Average trip length'].append(out['Average trip length'])
    res_dict['Average Job Surge'].append(out['Average Job Surge'])
    res_dict['Total GB'].append(out['Total GB'])
    res_dict['overwrite'].append(out['overwrite'])
    res_dict['Driver AR'].append(out['Driver AR'])
    res_dict['Rider cancel'].append(out['Rider cancel'])
    print(f'with coef {coef}:')
    print(out['match_rate'])

In [ ]:
df = df.reset_index()

In [ ]:
# # better with EFOF

df.iloc[6399]

In [ ]:
# better with FLOF

df.iloc[526]

In [ ]:
for i in range(500,600):
    ttt = df.iloc[i]
    if len(df[
        (df.supply_uuid == ttt.supply_uuid) & \
        (df.scan_uuid == ttt.scan_uuid)][['of_value_EFOF','new_of']].sort_values('new_of').head(15)) > 10:
        break

df[
        (df.supply_uuid == ttt.supply_uuid) & \
        (df.scan_uuid == ttt.scan_uuid)][['of_value_EFOF','new_of']].sort_values('new_of').head(15)


In [ ]:

# df[df.job_uuid == '9a031c82-6507-4cf6-b0eb-4bc74b42f3f3'][['of_value_EFOF','new_of']].sort_values('new_of').head(15)

df[
    (df.supply_uuid == '258ee6f9-d13f-4725-ae1d-f347c9453baa') & \
    (df.scan_uuid == '342b06d2-a7b0-4f3d-a687-0569b0d669af')][['of_value_EFOF','new_of']].sort_values('new_of').head(15)


In [ ]:
# for k in out['overwrite_summary']:
#     print(k, len(out['overwrite_summary'][k]))

# t = [item for k in out['overwrite_summary'] for item in out['overwrite_summary'][k]]
# t = out['overwrite_summary']['36ccf21a-38d3-40d9-a165-e3cec6276542']

# SUPS = [i[0] for i in t]
# JOBS1 = [i[1] for i in t]
# JOBS2 = [i[2] for i in t]

mean_vals = []

for k in out['overwrite_summary']:
    if out['overwrite_summary'][k] != []:
        JOBS1 = [i[1] for i in out['overwrite_summary'][k]]
        JOBS2 = [i[2] for i in out['overwrite_summary'][k]]
        p1 = df[(df.scan_uuid == k) & (df.job_uuid.isin(JOBS1))].dest_SV_mean.mean()
        p2 = df[(df.scan_uuid == k) & (df.job_uuid.isin(JOBS2))].dest_SV_mean.mean()
        if len(JOBS1) > 10:
            mean_vals.append([p1, p2])
            
# for k in out['overwrite_summary']:
#     if out['overwrite_summary'][k] != []:
#         JOBS = [i[0] for i in out['overwrite_summary'][k]]
#         SUPS1 = [i[1] for i in out['overwrite_summary'][k]]
#         SUPS2 = [i[2] for i in out['overwrite_summary'][k]]
#         p1 = df[(df.scan_uuid == k) &(df.job_uuid.isin(JOBS)) & (df.supply_uuid.isin(SUPS1))].origin_SV.mean()
#         p2 = df[(df.scan_uuid == k) &(df.job_uuid.isin(JOBS)) & (df.supply_uuid.isin(SUPS2))].origin_SV.mean()
#         if len(SUPS1) > 20:
#             mean_vals.append([p1, p2])
        

In [ ]:
mean_vals

In [ ]:
plt.figure()
plt.plot(res_dict['coef'], res_dict['match_rate'])
plt.xlabel('LTSV coef.')
plt.ylabel('match rate')

plt.figure()
plt.plot(res_dict['coef'], res_dict['Average Matched ETA'])
plt.xlabel('LTSV coef.')
plt.ylabel('Average Matched ETA')

plt.figure()
plt.plot(res_dict['coef'], res_dict['Average Matched Fare'])
plt.xlabel('LTSV coef.')
plt.ylabel('Average Matched Fare')

plt.figure()
plt.plot(res_dict['coef'], res_dict['Total GB'])
plt.xlabel('LTSV coef.')
plt.ylabel('Total GB')

plt.figure()
plt.plot(res_dict['coef'], res_dict['overwrite'])
plt.xlabel('LTSV coef.')
plt.ylabel('overwrite')

plt.figure()
plt.plot(res_dict['coef'], res_dict['Driver AR'])
plt.xlabel('LTSV coef.')
plt.ylabel('Driver AR')

plt.figure()
plt.plot(res_dict['coef'], res_dict['Rider cancel'])
plt.xlabel('LTSV coef.')
plt.ylabel('Rider cancel')